In [ ]:
# encoder takes input (some sentence)
# encodes to some CONTEXT z, 
# z is input to DecoderLSTM
# start with start token, and builds a sequence 

In [1]:
import sys 
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter



In [2]:
print(f'Python version: {sys. version}') 
print(sys. version_info)
print(f'torch ver:        {torch.__version__}')
print(f'torchtext ver:    {torchtext.__version__}')
print(f'spacy ver:        {spacy.__version__}')
print(f'numpy ver:       {np.__version__}')

Python version: 3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=18, releaselevel='final', serial=0)
torch ver:        2.1.0
torchtext ver:    0.6.0
spacy ver:        3.5.3
numpy ver:       1.24.3


In [5]:
# Download required:
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm

#spacy_ger = spacy.load("de_core_news_sm")
spacy_eng = spacy.load('en_core_web_sm')
spacy_ger = spacy.load('de_core_news_sm')
def tokenizer_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenizer_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]



In [6]:
german = Field(tokenize=tokenizer_ger, lower=True,
               init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenizer_eng, lower=True,
                init_token='<sos>',eos_token='<eos>')

train_data , validation_data, test_data = Multi30k.splits(exts=('.de','.en'),fields=(german,english))

german.build_vocab(train_data,max_size=10000,min_freq=2)
english.build_vocab(train_data,max_size=10000,min_freq=2)

downloading training.tar.gz


ConnectionError: HTTPConnectionPool(host='www.quest.dcs.shef.ac.uk', port=80): Max retries exceeded with url: /wmt16_files_mmt/training.tar.gz (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F6F62CBA30>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
class Encoder(nn.Module):
    